# **Phd Challenge : Coupled Thermo-hydraulic analysis of a 2-D Geothermal Reservoir**

## **This is the report about the new version of the challenge.**

## **Due to limitations of my computer, I was not able to implement meshes refined around the sources. Hence my simulations are not of the best quality.**

# **1. Introduction**

### **1.1 Enhanced vs. Conventional Geothermal Systems**

Enhanced Geothermal Systems (EGS) differ fundamentally from Conventional Geothermal Systems (CGS) in their approach to heat extraction [(source : Geothermal Power Plant)](https://roadmaps.mit.edu/index.php/Geothermal_Power_Plant):

- **Conventional Geothermal Systems (CGS)**: Extract heat from naturally occurring underground hydrothermal reservoirs where water at high temperature naturally circulates through permeable rock formations.

- **Enhanced Geothermal Systems (EGS)**: Artificially create or enhance permeability in hot, dry rock formations to enable fluid circulation. This process typically requires hydraulic fracturing from the injection well to create fluid pathways, which are then heated by the surrounding rock before being pumped back to the surface via the production well.

**Key Physical Differences:**

| Parameter | CGS | EGS |
|-----------|-----|-----|
| **Pore (Water) Pressure** | Pressure at water table (reservoir top) is atmospheric; reservoir pressure follows hydrostatic gradient | No natural water reservoir; pore pressure is roughly atmospheric everywhere, increasing locally when water is injected |
| **Temperature Distribution** | Uniform throughout reservoir due to high thermal conductivity of water | Follows geothermal gradient initially; cold injection water creates local temperature variations |  

<br></br>

<div style="text-align: center;">

![Geothermal systems](Images/geothermal_grids.png)

</div>


### **1.2 Symplifying assumptions from EGS Physics**

To simplify the modeling of EGS, we can make the following assumptions ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)):

- We assume that the **initial pressure** $p_0$ is **constant** throughout the domain, and equal to the atmospheric pressure.
- We assume a **geothermal gradient** in temperature, i.e.:

$$T_0(y) = T_s + G \cdot y$$

where : 
- $T_s$ is the surface temperature,
- $G$ is the geothermal gradient (in °C/m),
- $y$ is the depth (in m) below the surface.
- We assume the water and rock reach **thermal equilibrium**, i.e. the temperature of the water is equal to the temperature of the rock at any point in the reservoir.
- Rock is considered as homogeneous and isotropic, water is considered incompressible

Other symplifying assumptions are made, such as:
- The fluid (water here) is considered as a single phase (only liquid phase, no vapor phase).
- At the scale of the reservoir, the flow is considered laminar
- Rock is considered as homogeneous and isotropic, water is considered incompressible
- The properties are considered constant.
- The heat transfer is considered as conduction in the rock and convection in the fluid.

# **2. Thermo-Hydraulic model without fault**

## **2.1 Mathematical Formulation**

### **2.1.1 Governing Equations**

The coupled thermo-hydraulic evolution in porous media is described by equations derived from Biot's porothermoelasticity theory:

#### **Pressure equation**

$$\frac{\partial \Delta p}{\partial t} - c_p \nabla^2 \Delta p = \lambda_{pT} \frac{\partial \Delta T}{\partial t}$$

#### **Temperature equation**

$$\frac{\partial \Delta T}{\partial t} - c_T \nabla^2 \Delta T = \lambda_{Tp} \frac{\partial \Delta p}{\partial t}$$

**Parameters:**
- $c_p$, $c_T$: hydraulic and thermal diffusivities (m²·s⁻¹)
- $\lambda_{pT}$: mechano-caloric coupling coefficient (Pa·K⁻¹)
- $\lambda_{Tp}$: thermo-osmosis coupling coefficient (K·Pa⁻¹)

#### **Corrections from the subject**

$\lambda_{pT}$ and $\lambda_{Tp}$ are respectively assumed to be in Pa·K⁻¹·s⁻¹ and K·Pa⁻¹·s⁻¹, but obviously the s⁻¹ is a mistake and is removed.

#### **Parameter Variations**
All coefficients are assumed constant for simplicity. While it is acceptable for most coefficients, notable exceptions include:
- **Thermal diffusivity** $c_T$: varies significantly with temperature
- **Hydraulic diffusivity** $c_p$: varies significantly with high pore pressure

### **2.1.2 Problem Geometry and Boundary Conditions**

We consider a vertical 2D rectangular slice of the subsurface with dimensions chosen to ensure constant boundary conditions for pressure and temperature fields.

#### Characteristic Scales

**Well Scale:**
- Well radius: $r_w = 0.05-0.2$ m
- Flow rate: $Q_w = 0.05-0.2$ m³/s
- System diffusivities: $c_p$, $c_T$

**Reservoir Scale:**
- Well separation: $L = 500$ m
- Simulation duration: $t = 1$ year
- System diffusivities: $c_p$, $c_T$

#### Well Modeling Approach

To determine whether to model wells as line or point sources, we calculate the Péclet numbers:

$$Pe = \frac{Q_w}{c \pi r_w}$$

**Results:**
- Temperature Péclet number: $Pe_T \in [10^5, 10^7] \gg 1$
- Pressure Péclet number: $Pe_p \in [10^4, 10^6] \gg 1$

Since both Péclet numbers are much greater than 1, advection dominates diffusion in the wells, justifying **point source modeling** at well bottoms.

#### **Domain Size Determination**

In order to set the boundaries of the simulation box, we need to compute the characteristic diffusion distance of pressure and temperature.

**Assumption** : the left hand side of both equations are diffusion equations, the right hand side being coupling. Hence, **we assume the evolution to be of first-order** like diffusion equations. Thus, **we fix the stable region to be 10 times the characteristic diffusion distance**.

The characteristic diffusion distance is $\ell = \sqrt{c t}$ for simulation time $t = 1$ year:

- Pressure: $\ell_p = \sqrt{c_p t} = 177$ m → Boundary distance: $10\ell_p = 1770$ m
- Temperature: $\ell_T = \sqrt{c_T t} = 56$ m → Boundary distance: $10\ell_T = 560$ m

$1770~m$ being excessively large, an average boundary limit of $\boldsymbol{1000~m}$ was chosen.

#### **Boundary Conditions**

Following [(Gutiérrez-Oribio & Stefanou, 2024)](https://www.sciencedirect.com/science/article/pii/S2352380824000376) for drained pressure boundary conditions :
- **Left/Right boundaries**: No pressure flux $\frac{\partial p}{\partial x} = 0$, no temperature flux $\frac{\partial T}{\partial x} = 0$ 
- **Top boundary**: No pressure flux $\frac{\partial p}{\partial y} = 0$, constant temperature $T = T_s$
- **Bottom boundary**: No pressure flux $\frac{\partial p}{\partial y} = 0$, constant temperature $T = T_0$
- **Injection well**: Point source with pressure $p = p_{inj}$, temperature $T = T_{inj}$
- **Extraction well**: Point source with pressure $p = p_{ext}$

## **2.2. Dimensionless Formulation**

### **2.2.1 Including Point Sources**

Adding point sources for pressure and temperature:

#### Pressure Equation with Sources

$$\frac{\partial \Delta p}{\partial t} - c_p \nabla^2 \Delta p = \lambda_{pT} \frac{\partial \Delta T}{\partial t} + S_{ext}^p \delta(x-x_{ext})$$

#### Temperature Equation with Sources

$$\frac{\partial \Delta T}{\partial t} - c_T \nabla^2 \Delta T = \lambda_{Tp} \frac{\partial \Delta p}{\partial t} + S_{inj}^T \delta(x-x_{inj})$$

### **2.2.2 Dimensionless Variables**

**Characteristic Scales:**

| Quantity | Variable | Scale | Estimate |
|----------|----------|-------|----------|
| Length | $x, y$ | Well spacing $L$ | $L = 500$ m |
| Time | $t$ | Pressure diffusion time $t_c$ | $t_c = \frac{L^2}{c_p} = 2.5 \times 10^{8}$ s ≈ 8 years |
| Pressure change | $\Delta p$ | Pressure difference $\Delta p_c$ | $\Delta p_c = p_{inj} - p_{ext} = 3$ MPa |
| Temperature change | $\Delta T$ | Temperature difference $\Delta T_c$ | $\Delta T_c = T_0 - T_{inj} = 180$ °C |

**Dimensionless Equations:**

Using $\hat{\kappa} = \frac{c_T}{c_p}$ and $t_c = \frac{L^2}{c_p}$:

$$\begin{align}
\frac{\partial \hat{p}}{\partial \hat{t}} - \nabla_{\hat{x}}^2 \hat{p} &= \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} + \hat{S}_{inj}^p \hat{\delta}(\hat{x} - \hat{x}_{inj}) + \hat{S}_{ext}^p \hat{\delta}(\hat{x} - \hat{x}_{ext}) \\
\frac{\partial \hat{T}}{\partial \hat{t}} - \hat{\kappa} \nabla_{\hat{x}}^2 \hat{T} &= \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} + \hat{S}_{inj}^T \hat{\delta}(\hat{x} - \hat{x}_{inj})
\end{align}$$

**Dimensionless Parameters:**
$$\hat{\kappa} = \frac{c_T}{c_p}, \quad
\hat{\lambda}_{pT} = \lambda_{pT} \frac{\Delta T_c}{\Delta p_c}, \quad
\hat{\lambda}_{Tp} = \lambda_{Tp} \frac{\Delta p_c}{\Delta T_c}, \quad
\hat{S}_{inj}^p = \frac{1}{c_p \Delta p_c} \tilde{S}_{inj}^p, \quad
\hat{S}_{ext}^p = \frac{1}{c_p \Delta p_c} \tilde{S}_{ext}^p, \quad
\hat{S}_{inj}^T = \frac{1}{c_p \Delta T_c} \tilde{S}_{inj}^T
$$

## **2.3 Variational formulation**

### **2.3.1 Theoretical framework**

Let $\mathcal{B}$ be a body that initially occupies the compact domain $\Omega(\mathcal{B}) \in \mathbb{R}^n$ with boundary $\partial \Omega$. Since the domain is fixed, we assume that all configurations of the body coincide with this initial configuration, i.e., the geometry of the domain remains fixed and equal to $\Omega$.
We introduce two scalar fields defined over $\Omega$ : the **pore pressure** $p$ and the **temperature** $T$. We assume that both fields have square-integrable values and gradients, and thus belong to the first-order Sobolev space:
$$p, T \in H^1(\Omega) = \left \{\varphi \in L^2(\Omega) \left | \nabla \varphi \in \left[L^2(\Omega)\right]^n \right . \right \}$$ 


#### Boundary Conditions

The body is placed in an environment characterized by the following Dirichlet and Neumann boundary conditions :

**For Pressure:**
- $\partial \Omega = \partial \Omega_p^D \cup \partial \Omega_p^N$, $\partial \Omega_p^D \cap \partial \Omega_p^N = \emptyset$
  
- **Dirichlet**: $p = p_D$ on $\partial\Omega_p^D$
  
- **Neumann**: $\frac{\partial p}{\partial n} = q_p$ on $\partial\Omega_p^N$

**For Temperature:**
- $\partial \Omega = \partial \Omega_T^D \cup \partial \Omega_T^N$, $\partial \Omega_T^D \cap \partial \Omega_T^N = \emptyset$
  
- **Dirichlet**: $T = T_D$ on $\partial\Omega_T^D$
  
- **Neumann**: $\frac{\partial T}{\partial n} = q_T$ on $\partial\Omega_T^N$

#### Function Spaces

**Trial Spaces:**
$$V_p = \{p \in H^1(\Omega) : p|_{\partial \Omega_p^D} = p_d\}, \quad V_T = \{T \in H^1(\Omega) : T|_{\partial \Omega_T^D} = T_d\}$$

**Test Spaces:**
$$V^0_p = \{\delta p \in H^1(\Omega) : \delta p|_{\partial \Omega_p^D} = 0\}, \quad V^0_T = \{\delta T \in H^1(\Omega) : \delta T|_{\partial \Omega_T^D} = 0\}$$

**Mixed Spaces:**
$$V = V_p \times V_T, \quad V^0 = V^0_p \times V^0_T$$

### **2.3.2 Variational Formulation Development**

**Problem Statement:** At each time step $t$, find $(p(t), T(t)) \in V$ such that for all $(\delta p, \delta T) \in V^0$:
$$a((p(t), T(t)), (\delta p, \delta T)) = l((\delta p, \delta T))$$

with $a$ a bilinear form, $l$ a linear form, and the well-posedness conditionsare verified to apply the Lax-Milgram theorem which ensures a unique solution :

- $a(\cdot, \cdot)$ is continuous and coercive on $V$
- $l(\cdot)$ is continuous on $V$

Multiplying equations by test functions and integrating over $\Omega$:

$$\begin{align}
&\int_{\Omega} \left[\delta p \frac{\partial \hat{p}}{\partial \hat{t}} - \delta p \nabla_{\hat{x}}^2 \hat{p}\right] d\Omega = \int_{\Omega} \left[\delta p \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} \right] d\Omega + \hat{S}_{inj}^p \delta p(\hat{x}_{inj}) + \hat{S}_{ext}^p \delta p(\hat{x}_{ext}) \\
&\int_{\Omega} \left[\delta T \frac{\partial \hat{T}}{\partial \hat{t}} - \delta T \hat{\kappa} \nabla_{\hat{x}}^2 \hat{T}\right] d\Omega = \int_{\Omega} \left[\delta T \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} \right] d\Omega + \hat{S}_{inj}^T \delta T(\hat{x}_{inj})
\end{align}$$

#### Integration by Parts

Using integration by parts and noting that boundary terms vanish due to boundary conditions:

$$\begin{align}
&\int_{\Omega} \left[\delta p \frac{\partial \hat{p}}{\partial \hat{t}} + \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p}\right] d\Omega = \int_{\Omega} \left[\delta p \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} \right] d\Omega + \hat{S}_{inj}^p \delta p(\hat{x}_{inj}) + \hat{S}_{ext}^p \delta p(\hat{x}_{ext}) \\
&\int_{\Omega} \left[\delta T \frac{\partial \hat{T}}{\partial \hat{t}} + \hat{\kappa} \nabla_{\hat{x}} \delta T \cdot \nabla_{\hat{x}} \hat{T}\right] d\Omega = \int_{\Omega} \left[\delta T \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} \right] d\Omega + \hat{S}_{inj}^T \delta T(\hat{x}_{inj})
\end{align}$$

## **2.4 Dealing with singularities caused by the point sources**

The Dirac delta functions modeling point sources create singularities that pose significant problems :

- **Unbounded gradients** at source locations, leading to $\nabla p, \nabla T \to \infty$
- **Loss of regularity** in the solution, violating standard finite element assumptions
- **Numerical instabilities** due to poorly conditioned system matrices
- **Convergence issues** as mesh refinement doesn't improve accuracy near singularities

### **Possible solutions** (from [Béchet et al. (2019)](https://hal.science/hal-02136900/document))

#### **Gaussian Regularization**

Replace the Dirac delta with a smooth approximation (in 2D):

$$\delta_\varepsilon(x-x_{inj}) = \frac{1}{\varepsilon^2 \pi} \exp\left(-\frac{||x-x_{inj}||^2}{\varepsilon^2}\right)$$

**Implementation**:
1. Replace $\delta(x-x_{inj})$ with $\delta_\epsilon(x-x_{inj})$ in source terms
2. Choose $\epsilon$ to be $2h-5h$ typically
3. Use standard Gauss quadrature for integration

- **Advantages**: 
- Simple implementation
- It provides smooth solutions which are applicable in standard FEM without requiring special elements

- **Drawbacks**: 
- The solution depends on the artificial length scale $\epsilon$
- The mesh must be fine enough around the wells to resolve the gaussian decay


#### **Asymptotic Expansion Near Source**

Similarly to fracture analysis, the solution near a point source exhibits singular behavior that can be characterized analytically. The asymptotic expansion of weak discontinuities (continuous fields but discontinuous gradients) takes the form (in 2D):

$$p(r,\theta) = A_p \ln{r} + B_p + \mathcal{O}(r^2\ln{r})$$
$$T(r,\theta) = A_T \ln{r} + B_T + \mathcal{O}(r^2\ln{r})$$

The coefficients $A_p, A_T$ are related to the source strengths.

**Implementation - Barsoum-like Elements**:
1. Elements containing source points are identified
2. For quarter-point elements, mid-side nodes are moved (to quarter-points ($\xi = \pm 1/4$) for Barsoum)
3. Enforce continuity with surrounding standard elements

**Advantages**:
- It provides exact representation of singular behavior
- It enables optimal convergence rates

**Disadvantages**:
- It order to be rigorous, it requires analytical derivation of asymptotic forms which are non trivial for complex geometries
- It is quite complex to implement
- It requires very fine meshes around the sources


#### **Extended Finite Element Method (XFEM)**

They enrich standard finite element approximation with singular functions for the elements which contains the point sources:
$$p^h(x) = \sum_i N_i(x) p_i + \sum_j N_j(x) \sum_\alpha F_\alpha(x) b_j^\alpha$$

with $b_j$ the point sources and where enrichment functions for 2D sources are:
- $F_{\alpha} (r,\theta) = \ln{r} \mathcal{F}_{\alpha}(\theta)$

**Implementation**:
1. Elements containing sources are identified, and enriched by the specific shape functions
2. Special integration schemes are used for singular integrands

**Mesh requirements**:
- Standard mesh (no conformity to sources required)
- Enrichment zone around each source
- Fine mesh still recommended for accuracy

**Advantages**:
- No mesh conformity is required
- Modular implementation (add to existing FEM code)

**Disadvantages**:
- Additional degrees of freedom which increase system size
- Conditioning issues with enrichment ([Béchet et al. (2019)](https://hal.science/hal-02136900/document))
- The implementation is quite complex for coupled systems.

#### **J-integral like regularization**

Similarly to the J-integral in from fracture analysis, one could define a regularizing, path-independent energy functional associated with singularities without modeling them, which would provide precise evaluation of their effects away from the singularity.

**Implementation**
- Model a point source with a fine mesh
- Capture the far-field behaviour associated to the point source
- Use this asymptotic law on a circle to replace the point source in coarse-mesh simulations

**Acvantages:**
- It is path-independent
- It is physically sound
- It provides a stable numerical evaluation away from singularity

**Drawbacks:**
- Requires a two-step, complex simulation
- Path-independence may be lost with material heterogeneities
- May not be applicable for my system

## **2.5 Numerical Implementation: θ-Method**

### Inspired from [this example](https://github.com/kinnala/scikit-fem/blob/10.0.2/docs/examples/ex19.py) of Scikit-fem

### **2.5.1 Time Discretization**

For time step $\hat{\Delta t}$ between time levels $n$ and $n+1$:
$$\frac{\partial \hat{p}}{\partial \hat{t}} \approx \frac{\hat{p}^{n+1} - \hat{p}^n}{\hat{\Delta t}}$$

### **2.5.2 θ-Method**

**Runge-Kutta methods** could have been implemented. However, following [this example](https://github.com/kinnala/scikit-fem/blob/10.0.2/docs/examples/ex19.py) about heat equation of Scikit-fem, it was chosen to use the $\theta$-method. For $\theta \in [0, 1]$, spatial terms are approximated as:
$$\nabla_{\hat{x}} \hat{p} \approx \theta \nabla_{\hat{x}} \hat{p}^{n+1} + (1 - \theta) \nabla_{\hat{x}} \hat{p}^n$$

**Special Cases:**
- $\theta = 0$: **Explicit Euler** method
- $\theta = 1$: **Implicit Euler** method  
- $\theta = 0.5$: **Crank-Nicolson** method

### **2.5.3 Discretized Variational Formulation**

Multiplying by $\Delta \hat{t}$ and applying the θ-method:

$$\begin{align*}
&\int_{\Omega} \delta p (\hat{p}^{n+1} - \hat{p}^n) d\Omega + \theta \Delta \hat{t} \int_{\Omega} \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p}^{n+1} d\Omega + (1-\theta) \Delta \hat{t} \int_{\Omega} \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p}^n d\Omega \\
&= \hat{\lambda}_{pT} \int_{\Omega} \delta p (\hat{T}^{n+1} - \hat{T}^n) d\Omega + \Delta \hat{t} \left[ \hat{S}_{inj}^p \delta p(\hat{x}_{inj}) + \hat{S}_{ext}^p \delta p(\hat{x}_{ext}) \right] \\
&\int_{\Omega} \delta T (\hat{T}^{n+1} - \hat{T}^n) d\Omega + \theta \Delta \hat{t} \hat{\kappa} \int_{\Omega} \nabla_{\hat{x}} \delta T \cdot \nabla_{\hat{x}} \hat{T}^{n+1} d\Omega + (1-\theta) \Delta \hat{t} \hat{\kappa} \int_{\Omega} \nabla_{\hat{x}} \delta T \cdot \nabla_{\hat{x}} \hat{T}^n d\Omega \\
&= \hat{\lambda}_{Tp} \int_{\Omega} \delta T (\hat{p}^{n+1} - \hat{p}^n) d\Omega + \Delta \hat{t} \hat{S}_{inj}^T \delta T(\hat{x}_{inj})
\end{align*}$$

### **2.5.4 Matrix Formulation**

Define finite element operators:
- **Mass operator**: $\mathbb{M}_{v} u = \mathbf{M}(\delta v, u) = \int_{\Omega} \delta v \, u \, d\Omega$

- **Laplacian operator**: $\mathbb{L}_{v} u = \mathbf{L}(\delta v, u) = \int_{\Omega} \nabla_{\hat{x}} \delta v \cdot \nabla_{\hat{x}} u \, d\Omega$

The discretized system becomes:

$$\begin{bmatrix}
\hat{\mathbb{M}}_{p} + \theta \hat{\Delta t} \hat{\mathbb{L}}_{p} & -\hat{\lambda}_{pT}\hat{\mathbb{M}}_{p} \\
-\hat{\lambda}_{Tp} \hat{\mathbb{M}}_{T} & \hat{\mathbb{M}}_{T} + \hat{\kappa} \theta \hat{\Delta t} \hat{\mathbb{L}}_{T}
\end{bmatrix} 
\begin{bmatrix}
\hat{p}^{n+1} \\
\hat{T}^{n+1}
\end{bmatrix}$$
$$= 
\begin{bmatrix}
\hat{\mathbb{M}}_{p} - (1 - \theta) \hat{\Delta t} \hat{\mathbb{L}}_{p} & -\hat{\lambda}_{pT}\hat{\mathbb{M}}_{p} \\
-\hat{\lambda}_{Tp} \hat{\mathbb{M}}_{T} & \hat{\mathbb{M}}_{T} - \hat{\kappa} (1 - \theta) \hat{\Delta t} \hat{\mathbb{L}}_{T}
\end{bmatrix}
\begin{bmatrix}
\hat{p}^n \\
\hat{T}^n
\end{bmatrix}$$
$$+ \hat{\Delta t}
\begin{bmatrix}
\hat{S}_{inj}^p \hat{\mathbb{M}}_{p} \delta (\hat{x} - \hat{x}_{inj}) + \hat{S}_{ext}^p \hat{\mathbb{M}}_{p} \delta (\hat{x} - \hat{x}_{ext}) \\
\hat{S}_{inj}^T \hat{\mathbb{M}}_{T} \delta (\hat{x} - \hat{x}_{inj})
\end{bmatrix}
$$


## **2.6 Mesh, Time Discretization, and Solver Details**

### **2.6.1 Mesh Generation**

Initially, the computational mesh was generated using [Gmsh](https://onlinelibrary.wiley.com/doi/pdf/10.1002/nme.2579), with particular care taken to ensure accuracy near the injection and extraction wells. These source points are modeled as Dirac singularities, which require fine spatial resolution to properly capture steep local gradients in pressure and temperature fields.

To this end, a **locally refined mesh** was employed around the well locations. The rest of the domain was meshed more coarsely to reduce computational cost. **Second-order triangular elements** were used, which improve numerical stability and solution smoothness.

<div style="text-align: center;">

![Gmsh mesh](Images/mesh_gmsh.png)

</div>

### **Problem with refined mesh**

The mesh is **too refined for my laptop**. Hence, I was not able to use it, which means **I was not able to implement any of the regularization methods to deal with the point sources**. 

Instead, I used a regular, coarse mesh generated by Scikit-fem. This is **not wanted** because it can lead to insabilities and to unprecise behaviour caused by the point sources.

### **2.6.2 Time Stepping and Simulation Time**

The time step $\Delta t$ is selected based on both the diffusion CFL (Courant-Friedrichs-Levy) criterion and the characteristic thermal diffusion time scale:
$$
\Delta t < \frac{h^2}{4 c_p}
$$
where $h$ is the local mesh size near the wells and $c_p$ is the hydraulic diffusivity. This ensures that both pressure and temperature fields evolve smoothly without introducing spurious oscillations.

With a mesh size of $78.1~ m$, we obtain a maximum time step $\Delta t_{max} = 17.6$ days.

### **2.6.3 Linear Solver Setup**

The coupled thermo-hydraulic system is assembled as a block matrix capturing mass, diffusion, and cross-coupling terms. A sparse LU decomposition is used to solve the Crank–Nicolson system efficiently at each time step.

Wells were initially treated as fixed-value point sources, with their values enforced after each solve to ensure stability. However it lead to strong instabilities.
Hence, they were later treated as fixed Dirichlet conditions. Dirichlet boundary conditions are applied similarly.

### **2.7 Results and analysis**

Herebelow are the results of temperature (a-d) and pressure (e-h) after 1, 30, 180 and 365 days respectively.

<div style="text-align: center;">

![Simulation plots](Images/simulations.png)

</div>

**Pressure evolution**

It is also striking that pressure diverges immediately, leading to almost 6MPa in the first iteration despite the injection pressure being fixed to 2MPa. This has one main cause : the mesh is too coarse, despite the CFL condition being fulfilled.

However, pressure diffusion seems acceptable, despite a pressure well of -1Mpa around the injection which is likely an artifact of the pressure gradient capping.

**No temperature change**

It is clearly noticeable there is no temperature change. This has one main causes :
- the thermo-osmosis coefficient is 200 times inferior to the mechano-caloric coefficient, leading to a very weak thermal diffusion

**Steady state**

The fields' evolution does not seems to reach a steady state at the end of the simulation. This is confirmed by plotting the evolution herebelow, which shows very clearly that the evolution is not stabilizing. This can be explained by the constants not being large enough and by the gradient capping.

![Steady state](Images/T_P_graphs.png)

**Domain size and mesh resolution**

An obvious remark is that **mesh resolution is too coarse**, leading to somewhat absurd results. However, my computer cannot handle finer meshes due to limited computational resources.

# **3. Conceptual Fault Extension**

## **3.1 Variational Formulation**

We use the framework developed in **2.3**, and add a fault $\Gamma$ (oriented surface of dimension $dim(\Omega) - 1$) in the formulation. The test and trial spaces stay the same provided they can account for the discontinuity. We define the unit normal to the interface $\vec{n}$, and define the jump operator across the fracture :

$$[[ u ]] = u^+ - u^-$$

Since the domain, including the fracture $\Gamma$, do not change in time and the fields $p$ and $T$ are piecewise continuous (i.e. continuous on each side of the interface), **time derivatives do not need an additional term**. Only the spatial derivatives need it. 

Let's reuse the nondimensionalized equations, neglecting the source terms for simplicity. Multiplying by the test functions and integrating over $\Omega$ :

$$\begin{align*}
&\int_{\Omega} \left[\delta p \frac{\partial \hat{p}}{\partial \hat{t}} - \delta p \nabla_{\hat{x}}^2 \hat{p}\right] d\Omega = \int_{\Omega} \left[\delta p \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} \right] d\Omega \\
&\int_{\Omega} \left[\delta T \frac{\partial \hat{T}}{\partial \hat{t}} - \delta T \hat{\kappa} \nabla_{\hat{x}}^2 \hat{T}\right] d\Omega = \int_{\Omega} \left[\delta T \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} \right] d\Omega
\end{align*}$$

Let's integrate by parts one spatial term to understand :

$$\int_{\Omega} \delta p \cdot \nabla_{\hat{x}}^2 \hat{p} d\Omega = \int_{\partial \Omega} \delta p \cdot \left( \nabla_{\hat{x}} \hat{p} \cdot \vec{n} \right) dS - \int_{\Omega \backslash \Gamma} \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p} d\Omega + \int_{\Gamma^+} \delta p \cdot \left( \nabla_{\hat{x}} \hat{p} \cdot \vec{n}^+ \right) d\Gamma + \int_{\Gamma^-} \delta p \cdot \left( \nabla_{\hat{x}} \hat{p} \cdot \vec{n}^- \right) d\Gamma$$

The integral on $\partial \Omega$ is 0 due to boundary conditions,. Since $\vec{n}^- = -\vec{n}$ and $\vec{n}^+ = \vec{n}$, we then have : 
$$\int_{\Gamma^+} \delta p \cdot \left( \nabla_{\hat{x}} \hat{p} \cdot \vec{n}^+ \right) d\Gamma + \int_{\Gamma^-} \delta p \cdot \left( \nabla_{\hat{x}} \hat{p} \cdot \vec{n}^- \right) d\Gamma = \int_{\Gamma} \delta p \cdot [[  \nabla_{\hat{x}} \hat{p} \cdot \vec{n} ]] d\Gamma$$

Doing so with all the spatial terms of the TH equations, we get : 

$$\begin{align*}
&\int_{\Omega} \left[\delta p \frac{\partial \hat{p}}{\partial \hat{t}} + \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p}\right] d\Omega = \int_{\Omega} \left[\delta p \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} \right] d\Omega + \int_{\Gamma} \delta p \left( [[ \nabla_{\hat{x}} \hat{p} \cdot \vec{n} ]] \right) d \Gamma\\
&\int_{\Omega} \left[\delta T \frac{\partial \hat{T}}{\partial \hat{t}} + \hat{\kappa} \nabla_{\hat{x}} \delta T \cdot \nabla_{\hat{x}} \hat{T}\right] d\Omega = \int_{\Omega} \left[\delta T \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} \right] d\Omega + \int_{\Gamma} \delta T \left( [[ \nabla_{\hat{x}} \hat{T} \cdot \vec{n} ]] \right) d \Gamma
\end{align*}$$

Then, in order to deal with the jump terms, one often postulates a constitutive equation of the interfacial terms of the form : 

$$[[ \nabla_{\hat{x}} \hat{p} \cdot \vec{n} ]] = \hat{\alpha}_p [[ \hat{p} ]]$$

and thus gets : 

$$\begin{align}
&\int_{\Omega} \left[\delta p \frac{\partial \hat{p}}{\partial \hat{t}} + \nabla_{\hat{x}} \delta p \cdot \nabla_{\hat{x}} \hat{p}\right] d\Omega = \int_{\Omega} \left[\delta p \hat{\lambda}_{pT} \frac{\partial \hat{T}}{\partial \hat{t}} \right] d\Omega + \int_{\Gamma} \delta p \left(\hat{\alpha}_p [[ \hat{p} ]] \right) d \Gamma\\
&\int_{\Omega} \left[\delta T \frac{\partial \hat{T}}{\partial \hat{t}} + \hat{\kappa} \nabla_{\hat{x}} \delta T \cdot \nabla_{\hat{x}} \hat{T}\right] d\Omega = \int_{\Omega} \left[\delta T \hat{\lambda}_{Tp} \frac{\partial \hat{p}}{\partial \hat{t}} \right] d\Omega + \int_{\Gamma} \delta T \left(\hat{\alpha}_T [[ \hat{T} ]] \right) d \Gamma
\end{align}$$

## **3.2 Numerical methods**

This paragraph may seem similar to **2.4 Dealing with singularities caused by the point sources**. However, here, we deal with linear discontinuities and not with point singularities, hence most methods presented above do not apply.

There are several methods which can model faults in numerical modeling. They can be classified into three main groups : **equivalent volume continuum approaches**, **explicit volume fracture modeling approaches** and **surface methods**.


### **Volume methods**

Volume methods map the a volume mesh. As volume meshes, they suffer from the **artificial truncation** of the domain [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf). Indeed, the domain must be limited. If limited too close to the region of interest (ROI), **boundary effects** appear and disturb the solution. However, the **curse of dimensionality** makes it complicated to place the boundaries too far from the ROI since their complexity if of the order of $\mathcal{O}(N^3)$.

### **3.2.1 Equivalent continuum approaches**

These methods simplify the representation of fractured media by homogenizing their properties into an equivalent continuum, effectively "smearing" the fracture effects over a larger domain.

**Equivalent Continuum Model (ECM)**

In ECM, fractures are modeled implicitly by modifying the hydro-mechanical properties (e.g., equivalent permeability and stiffness tensors) of a continuum model to take into account fracture properties of fractures. The characteristics of the fractures are then homogeneized by this continuum ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

It offers the lowest computational cost, making it attractive for large-scale simulations. It is appropriate for modeling short cracks where explicit detailing of each fracture is not computationally feasible or cost-effective. Two major limitations are its inability to accurately consider mass transfer between the matrix and individual fractures explicitly, and the lack of explicit distinction between the fracture network and the rock matrix, making it difficult to represent the preferential flow paths in fractures ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

**Dual-Continuum Model (DCM)**

DCM models the fractured porous medium as two overlapping continua: one representing the rock matrix and the other representing the fracture network. Each continuum is assigned its own porosity and permeability, and they are coupled by a transfer function that describes flux exchange between them. This is important since the permeability of the fault can be up to 10⁵ higher than the permeability of the matrix ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400); [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

It offers too a low computational cost and is better at modeling fractures than ECM. However, it suffers from the same drawbacks ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).


**Phase-Field Methods**

Phase-field methods use a diffuse interface representation where fractures are characterized by a damage parameter $\phi \in [0,1]$, providing mesh-independent crack modeling. They enable natural crack nucleation and propagation, can handle complex crack patterns and are thermodynamically consistent. However, they cannot model sharp jumps and suffer from an important dependence on numerical parameters. Finally, they need very fine meshes in order to resolve the cracks properly, making them computationally prohibitive [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846).

### **3.2.2 Explicit fracture modeling approaches**

These methods represent the fault as a distinct geometric entity within the computational domain, possessing its own specific properties and governing equations, embedded within or interacting with the surrounding rock matrix.

**Finite differences**

In finite differences, operators are approximated by summations by part and penalty terms are used to weakly enforce boundary conditions [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf). They are efficient for regular meshes, but cannot model nonregular geometries. 

**Discontinuous Galerkin (DG)**

In DG, jump conditions are taken into account in every node, naturally handling the propagation and nucleation of displacement discontinuities everywhere with local conservations properties, making them highly physical [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf). Furthermore, they have excellent parallelization properties. However, the mesh has to resolve the characteristic length of cracks and cracks may appear everywhere, making it less cntrollable. Finally, its higher memory usage makes it less scalable.

**Discrete Fracture Model (DFM)**

DFM is a very close model to the variational formulation. In DFM, fractures are explicitly represented as lower-dimensional objects embedded within a higher-dimensional matrix. Fluid flow within these fracture elements is typically governed by the cubic law, and this flow is coupled to Darcy flow in the surrounding matrix via leak-off terms ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

DFM offers high accuracy in representing the precise geometry and flow characteristics of individual fractures. It explicitly models mass and heat transfer between the fracture and matrix, and can capture localized phenomena such as aperture changes and preferential flow paths ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

However, a significant drawback of DFM is the requirement for a detailed, transient mesh that conforms precisely to the location and geometry of the fractures. This often necessitates highly refined meshes in order to align with the fault planes, particularly for complex or dynamically evolving fracture networks. Geometric constraints imposed by complex fault geometries can also make mesh generation extremely challenging and lead to aberrations ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

**eXtended Finite Element Method (XFEM)**

XFEM is an extension of the standard FEM that allows for the representation of discontinuities without requiring the mesh to conform to the discontinuity's geometry. This is achieved by enriching the standard finite element approximation functions locally near the discontinuity using a partition of unity concept ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846) ; [Béchet et al., 2019](https://hal.science/hal-02136900/document)).

In the variational formulation using XFEM ([ETH course](https://ethz.ch/content/dam/ethz/special-interest/baug/ibk/structural-mechanics-dam/education/femII/XFEM.pdf)):

- **Displacement Field**: A strong discontinuity (a jump in the field variable) across the fracture is handled by introducing enrichment functions with Heavyside functions for nodal points whose support is bisected by the discontinuity. This allows for explicit representation of fault slip or dilation.
  $$u^h(x) = \sum_i N_i(x) u_i + \sum_j N_j(x) H(x) a_j + \sum_k N_k(x) \sum_\alpha F_\alpha(x) b_j^\alpha$$
  with $H$ the Heavyside function which models the discontinuity at the crack, and $F \propto \sqrt{r}$ the asymptotic expansion at the crack tip

- **Pressure and Temperature Fields**: For these fields, a weak discontinuity is typically assumed, meaning the fields themselves are continuous across the fracture, but their gradients are discontinuous. This behavior is captured by enriching the approximation fields using a signed distance function.
  $$p^h(x) = \sum_i N_i(x) p_i + \sum_j N_j(x) \phi(x) a_j + \sum_k N_k(x) \sum_\alpha G_\alpha(x) b_j^\alpha$$
  with $\phi$ the signed distance function which models the gradient discontinuity at the crack, and $G \propto \ln{r}$ the asymptotic expansion at the crack tip

XFEM accounts explicitly for mass echange and between the fracture and the matrix at the enriched nodes, along with fracture flow in the enriched elements. Its ability to model complex fracture paths without the need of a conforming mesh is a very strong point.

However, this comes at the expense of an increased number of degrees of freedom in in the enriched elements, along with interfacial integrations for coupling terms ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)). However, this is restricted to the elements which are intersected by the discontinuities.  
Furthermore, it can lead to ill-condiitioned matrices ([Béchet et al., 2019](https://hal.science/hal-02136900/document)).

**Finite-Discrete Element Method**

FDEM is a hybrid method that merges the benefits of FEM and DEM. It also incorporates advantages from cohesive element models for simulating fracture propagation, which is a strong incentive in EGS ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

In FDEM, the fracture-pore mixed seepage problem is converted into pore seepage within tetrahedral elements, fluid exchange through unbroken joint elements, and fracture seepage in broken joint elements (representing fractures). FDEM is particularly well-suited for simulating solid fracturing and dynamic fracture propagation, which is highly relevant in EGS. While powerful, hydro-mechanical coupling and fluid-driven cracking imply considerable complexity in the implementation and solution of FDEM models ([Yan et al., 2022](https://www.sciencedirect.com/science/article/pii/S0955799722001400) ; [Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)).

### **Surface methods**

Surface methods reformulate the volume PDE as a boundary integral equation (BIE), based on the fundamental solution of the system. A massive advantage is that they only require the discretization of the fault surface, reducing the complexity of the model by one dimension, being of complexity $\mathcal{O}(N^2)$ for standard BEM [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf). Furthermore, no domain truncation is needed, effectively cancelling the boundary effects and naturally incorporating far-field conditions through the fundamental solution.
The main disadvantage is that, for complex problems, the fundamental solution may be very difficult to find or may not exist analytically. For material inhomogeneities, a simple BIE formulation may not be available, requiring numerical or semi-analytical fundamental solutions. Furthermore, standard BEM produces dense matrices, leading to $\mathcal{O}(N^2)$ computational efficiency, though this is mitigated by fast BEM [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf).

**Spectral BEM (S-BEM)**

S-BEM uses FFT to accelerate computation **for planar fault geometries**. The boundary integral simplifies to a Hilbert transform in Fourier space: 
$$\tilde{\tau}(k) = \frac{\mu}{2} |k| \tilde{u}(k)$$
where $\tilde{\tau}(k)$ and $\tilde{u}(k)$ are the Fourier transforms of shear stress and displacement jump, respectively, $\mu$ is the shear modulus, and $k$ is the wavenumber. 

This reduces the computation from $\mathcal{O}(N^2)$ to $\mathcal{O}(N\ln{N})$, making it extremely efficient [(L. Bagur, 2024)](https://theses.hal.science/tel-04791294v1/file/123923_BAGUR_2024_archivage.pdf).

**Fast BEM**

Fast BEM methods like **Fast Multipole Method** or **H-matrix** achieve $\mathcal{O}(N\ln{N})$ complexity like S-BEM while handling **non-planar, irregular fault geometries**. However, they are very complex to implement.

### **3.2.4 Hybrid approaches**

**XFEM-ECM**

This approach leverages XFEM for modeling large-scale fractures to accurately capture mass and heat transfer at these critical discontinuities. Simultaneously, ECM is applied to represent networks of small-scale fractures where explicit modeling of every detail is computationally prohibitive ([Mortazavi et al., 2023](https://www.sciencedirect.com/science/article/pii/S1359431123007846)). Furthermore, while the position and orientation of large-scale fractures is known, small-scale fractures are more complicated to precisely model. By homogeneizing them, ECM allows to not take these limitations into account.

**FEM-BEM**

This approach leverages FEM for modeling bulk processes in inhomogeneous media, while BEM manages fracture interfaces and fracture effects. Such a scheme is highly efficient, however BEM matrices are often ill-conditioned, requiring specific techniques to handle. Furthermore, FEM-BEM interface must be done carefully.

### **3.2.6 Comparative analysis**

| **Method** | **Complexity** | **Memory** | **Mesh requirements** | **Fracture geometry** | **THM coupling** |
|-------|---------|-------|---------|--------|---------|
| ECM/DCM | $\mathcal{O}(N^3)$ | Low | Regular | Homogenized | Limited |
| Phase fields | $\mathcal{O}(N^3)$ | High | Fine | Diffuse | Good |
| DG | $\mathcal{O}(N^3)$ | Very high | Very fine | Arbitrary, dynamic | Good |
| DFM | $\mathcal{O}(N^3)$ | High | Conforming | Arbitrary | Good |
| XFEM | $\mathcal{O}(N^3)$ | Regular | Non-conforming | Arbitrary | Very good |
| FDEM | $\mathcal{O}(N^3)$ | High | Adaptative | Arbitrary, dynamic | Good |
| S-BEM | $\mathcal{O}(N \ln{N})$ | Medium | Surface only | Planar only | Limited |
| Fast BEM | $\mathcal{O}(N \ln{N})$ | Medium | Surface only | Arbitrary | Limited |

### **3.2.5 Modeling an oblique fault**

Here, we assume a large-scale, straight oblique fault. Continuum approaches are then not suited, as they would not take into account properly the leak-off and fluid flow through the fracture. Among the three possible explicite fracture modeling approaches, I would choose :

- **FDEM** if we want to model accurately the propagation of the fault under large pressure.
- **XFEM** if we want a mesh-independent mesh which does not suffer from the drawbacks of conforming meshes.
- **S-BEM** if we want a computationally efficient technique.

## **3.3 Physical effects of the fault**

The presence of an oblique normal fault directly impacts pressure drawdown, thermal breakthrough, and overall heat-extraction performance. The inclination and roughness of the fault have a big influence on the breakthrough.

Due to the permeability of the faults being up to 10⁵ that of the matrix, its properties must be studied very carefully. ([Lv et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522001134))

**Pressure drawdown**

If the fault is highly permeable (e.g., due to a large aperture or enhanced permeability from shear dilation), it can act as a preferential flow path, leading to rapid fluid movement along its plane. This can result in a faster pressure drop between injection and production wells if the fault connects them efficiently. For example, if the fault's inclination aligns closely with the injection-production (I-P) diagonal, the working fluid flows along this alignment, traveling a shorter distance and resulting in a higher rate of water production. Conversely, if the fault acts as a barrier (e.g., due to mineralization), it can impede fluid flow, leading to higher pressure buildup on the injection side and reduced pressure drawdown on the extraction side. ([Lv et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522001134), [Okoroafor et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522000086))

Furthermore, an increase in fluid pressure can lead to fault dilation. This, in turn, enhances its permeability (via the cubic law) and leads to a faster pressure drawdown.

**Thermal breakthrough**

Thermal breakthrough, the point at which cold injected fluid reaches the production well, causing a sharp drop in extracted fluid temperature, is highly sensitive to the fault's presence and characteristics.

If the oblique normal fault acts as a high-permeability path and is aligned with the I-P diagonal, it can create a short-circuit between the injection and production wells. This allows the cold injected fluid to bypass a larger volume of hot rock, leading to premature thermal breakthrough and a significant reduction in the efficiency of heat extraction. ([Lv et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522001134), [Okoroafor et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522000086))
However, if the fault is perpendicular to the I-P diagonal, it will act like a barrier, delaying the thermal breakthrough.
Finally, there is a lower flow-wetted surface in I-P faults with respect to perpendicular faults ([Okoroafor et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522000086)). That means I-P faults result in a lower heat extraction performance, leading to an even faster thermal breakthrough.

Furthermore, a larger fracture aperture, while increasing flow rate, can also reduce the flow-wetted surface area per unit volume of fluid, limiting heat transfer from the rock to the fluid. However, a surface fracture with higher spatial variations in aperture can result in greater heat extraction due to a larger surface area for heat transfer ([Okoroafor et al., 2022](https://www.sciencedirect.com/science/article/pii/S0375650522000086)).

<div style="text-align: center;">

![Effect of faults](Images/faults.png)

</div>

# **4. Controlling the system**

Controlling this system is inherently challenging. Indeed, the **known dynamics** captured by the numerical model only approximate the real behavior, and in practice, only the **input and output variables** of the physical system measured at the wellbores are observable, which makes control particularly difficult.

To overcome the computational cost of using the full numerical model online, we first introduce a **Reduced-Order Model (ROM)** (for example an input-convex neural network) trained offline on high-fidelity simulation data. This ROM is then used to feed a controller, in that case a **model predictive controller** ([Daniilidis et al, 2017](https://www.sciencedirect.com/science/article/pii/S0306261917308036)).

However, due to limited observability and model inaccuracies, any control policy may drift from reality over time. To maintain performance and constraint satisfaction, we introduce an **adaptive learning layer** based on **Reinforcement Learning (RL)** —e.g., a DDPG agent as in [Gutiérrez-Oribio et al. (2024)](https://onlinelibrary.wiley.com/doi/10.1002/nag.3923)— to periodically adjust the controller’s parameters.

## **4.1 Definition of the system inputs, external inputs and system outputs**

### **4.1.1 System inputs**

These are the control variables directly affecting the system.

As stated in the variational formulation, **the actual system input is the fluid injection flux** $q_i$, represented by $S_{inj}^p$ in the variational formulation.

The input pressure $p_i$ is a consequence of the model (hence a **system output**), since it is linked by Darcy's law to $q_i$ : $q_i = -\frac{K}{\mu} \nabla p_i$.

### **4.1.2 System outputs**

These are observable, uncontrolled quantities used to evaluate performance:

- **Output temperature** $T_e$ measured at the extraction well.
- **Output pressure** $p_e$ measured at the extraction well.
- **Output flux** $q_e$ measured at the extraction well. Numerically, it can be accessed via Darcy's law from the output pressure $p_e$.
- **Input pressure** $p_i$ measured at the injection well.
- **Seismicity rate** SR, needed to prevent seismicity events [(Gutiérrez-Oribio & Stefanou, 2024)](https://www.sciencedirect.com/science/article/pii/S2352380824000376). Numerically, it is implemented by the model developed by Gutierrez.

### **4.1.3 External inputs**

These are uncontrolled inputs affecting the system behaviour:

- **Power demand** $D(t)$ : the required power output. This is an external input that the control system must track.
- **Injection temperature** $T_i$ constant at $20^\circ C$.
- **Model uncertainties**.

### **4.1.4 Power function** $\boldsymbol{W(q_e, T_e)}$

The power function, which is tracked on the demand, is a nonlinear function of the output flux and temperature : $W = f(q_e, T_e)$. A simple nonlinear approximation is given by $$W = \rho c_p (T_e - T_{min})$$ 

with $T_{min}$ the minimum operating temperature. However, according to [Daniilidis et al, 2017](https://www.sciencedirect.com/science/article/pii/S0306261917308036), strong variations in the power output can change the chemical composition of the reservoir, which can in turn clog it and accelerate the thermal breakthrough and pressure drawdown. 

Hence, storage devices can be use to smooth out the demand $D(t)$. In turn, by noting $(T^*_e, q^*_e)$ the operating point of the system, one can linearize the power output :
$$W \approx q^*_e (T^*_e - T_{min}) + (T^*_e - T_{min}) \Delta p_e + q_e^* \Delta T_e$$

### **4.1.5 Constraints on the system**

The following constraints on the system are :

**Hard constraints** that must always be met :
- **Minimal operating temperature** $T_e > T_{min}$. One can define a safety margin $\delta T$, and by posing $r_T = T_{min} + \delta T$, we obtain $T_e > r_T$. This is a **threshold** to never go under. Such a threshold can be enforced by the penalty $P_T = max(0, r_T - T_e)$ which will activate when the temperature gets below $r_T$.
- - **Minimal operating pressure differential** $\Delta p = p_e - p_i > \Delta p_{min}$. One can define a safety margin $\delta \Delta p$, and by posing $r_p = \Delta p_{min} + \delta \Delta p$, we obtain $\Delta p > r_p$. This is a **threshold** to never go under. Such a threshold can be enforced by the penalty $P_p = max(0, r_p - \Delta p)$ which will activate when the pressure differential gets below $r_p$.

**Soft constraints**. These are typically enforced with a **tracking objective** :
- **Power demand** $W(t)$ We want to minimize the error $e_W(t) = (W(t) - D(t))$.
- **Seismicity rate** $SR$. We want to minimize the error $e_{SR}(t) = SR(t) - 1$.
- **Variations of the input flux** $|| \dot{q_i} ||^2$. We want to minimize the error $e_q = || \dot{q_i} ||^2$ to avoid unstable policies and chemical changes.

## **4.2 Robust control loop for EGS**

### **4.2.1 Analytical method : Adjoint problem**

For an optimal control problem with state equation constraints, we formulate the Lagrangian:

$$\mathcal{L}(u, \lambda, p) = \mathcal{J}(u,p) + \int_{\Omega} \lambda \cdot \mathcal{R}(u, p) d\Omega$$

where:
- $u$ is the state variable (here pressure, temperature)
- $p$ is the control parameter (extraction well temperature)
- $\lambda$ is the adjoint variable (Langrange multiplier)
- $\mathcal{R}(u, p) = 0$ represents the PDE
- $\mathcal{J}(u,p)$ is the objective function.

**Optimality conditions:**
$$\begin{align*}
\frac{\partial \mathcal{L}}{\partial \lambda} = 0 &\quad \Rightarrow \quad \mathcal{R}(u, p) = 0 \quad \text{(Direct problem)} \\
\frac{\partial \mathcal{L}}{\partial u} = 0 &\quad \Rightarrow \quad \frac{\partial \mathcal{J}}{\partial u} + \int_\Omega \lambda \frac{\partial \mathcal{R}}{\partial u} \, dx = 0 \quad \text{(Adjoint problem)} \\
\frac{\partial \mathcal{L}}{\partial p} = 0 &\quad \Rightarrow \quad \frac{\partial \mathcal{J}}{\partial p} + \int_\Omega \lambda \frac{\partial \mathcal{R}}{\partial p} \, dx = 0 \quad \text{(Optimality condition)}
\end{align*}$$

**Solution procedure:**

1. Solve forward problem: $\mathcal{R}(u^*,p) = 0$ for a given $p$
2. Solve adjoint problem: $\left( \frac{\partial \mathcal{R}}{\partial u} \right)^T \lambda^* = - \frac{\partial \mathcal{J}}{\partial u}$
3. Update control: $\frac{\partial \mathcal{J}}{\partial p} = - \int_\Omega \lambda^* \frac{\partial \mathcal{R}}{\partial p} d\Omega$

#### **Drawbacks**
- It is fully based on the PDE which doesn't represent fully the system
- It is difficult to take model errors into accounts and constraints may be less natural to enforce
- It requires to solve **both** the direct and ajoint problem at each step, adjoint problem which is often complex


### **4.2.2 Controller**

Instead, in order to enforce the constraints, one can use :
- A classical controller such as a **proprotional-integral-differential** (PID) controller. These controllers guarantee stability, but cannot guarantee constraints are always met ([Daniilidis et al, 2017](https://www.sciencedirect.com/science/article/pii/S0306261917308036))
- A modern **model predictive controller** (MPC) which does have the capacity to guarantee that constraints are always statisfied, but proving their stability is difficult.


Hence, following [Daniilidis et al, 2017](https://www.sciencedirect.com/science/article/pii/S0306261917308036), we choose a MPC to control the system. A MPC enforces constraints and optimizes performance over a finite horizon $N_p$. For computational efficiency, it uses the ROM to calculate the solutions. The finite horizon $N_p$ must be chosen accordingly to the characteristic time of the system and the uncertainties on the evolution of the system. At each time step $k$, the MPC solves :

$$(q_{i,k}, \cdots, q_{i, k+N_p}) = \min\limits_{q_{i,k}, \cdots, q_{i, k+N_p}} J, \quad J = \sum_{j=0}^{N_p} \left[ \alpha_w e_{W, k+j|k} + \alpha_T t_{T, k+j|k} + \alpha_p e_{p, k+j|k} + \alpha_{SR} e_{SR, k+j|k} + \alpha_q e_{q, k+j|k}\right]$$

with $\alpha_w, \alpha_T, \alpha_p, \alpha_{SR}, \alpha_q$ the gains of the controller. The gains related to hard constraints $\alpha_T$ and $\alpha_p$ are significantly higher than the others to ensure they are respected.

#### **Implementing adjoint problem in the MPC**

The adjoint problem predicts very well the known behaviour. Hence, it is possible to implement it in the minimization process of the MPC : it will provide a high-fidelity solution for the known behaviour, which will then be corrected by the constraints of the MPC.

### **4.2.3 Adaptive learning**

Due to model errors and unobserved dynamics, the MPC's policy may degrade over time. An **adaptive learning layer** adjusts the MPC parameters periodically to improve long-term performance. It takes in input the outputs of the system and the gains of the controller, and outputs new gains for the controller. Following [Gutiérrez-Oribio et al. (2024)](https://onlinelibrary.wiley.com/doi/10.1002/nag.3923), one could implement a RL layer with the following features :

- The **RL agent** could be a DDPG, well suited to continuous action spaces.
- The **Reward function** could be designed to maximize cumulative long-term thermal energy extraction and adherence to demand, while heavily penalizing constraint violations, and penalizing temperature and pressure differentials which stay too long too close from their threshold and high control variations.

## **4.3 Numerical implementation**

The numerical implementation developed in section one would be changed as follows :

- The **inputs/outputs** of the system would be changed. The source point $S_{inj}^p$ would be the control input, and the input/output pressures would be determined by Darcy's law from the respective associated fluid fluxes.
- A **Reduced-Order Model** (ROM) of the numerical model would be implemented in order to speed up the control loop.
- A **MPC** would optimizes $S_{inj}^p$ over a prediction horizon using the ROM.
- Finally, the **DDPG agent** would supervise the system, adapting the control policy periodically for robustness.

<div style="text-align: center;">

![Closed-loop control system](Images/Control_loop.png)

</div>

Together, this closed-loop system would ensure the long term objectives would be met.